In [ ]:
import sys
sys.path.append(f'/home/jselvaraaj_umass_edu/applications-of-NDE')

In [ ]:
!pip install gymnasium

!pip install wandb

#Setting up pygame for colab
!python -m pip install pygame

# Graphics and plotting.
print('Installing mediapy:')
!apt update && apt install -y ffmpeg
!command -v ffmpeg >/dev/null
!pip install -q mediapy

#for neural CDE
!pip install torchcde torchdiffeq

In [ ]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

import mediapy as media
import matplotlib.pyplot as plt

import torchcde
import torch

In [ ]:
!python main.py

In [ ]:
!python main.py baseline=False

In [ ]:
!python main.py

In [ ]:
!python main.py baseline=False